# **Evolution to a small-world network with chaotic units**

## *Import Libraries*

In [1]:
from ast import Return
import numpy as np
import random
import matplotlib.pyplot as plt
import networkx as nx
import os
import seaborn as sns; sns.set_theme()
from pathlib import Path

## *Auxiliary Functions*

In [2]:
def CLUSTERS(x,delta,i):
    y=x
    CLUSTER = [] #x inmutable, vario el arreglo CLUSTER
    CLUSTER.append(x[i])
    y=np.delete(y,int(x[i]))
    for j in range(len(x)):
        dij = abs(x[i] - x[j])
        if dij < delta and i != j:
            CLUSTER.append(x[j])
            y=np.delete(y,int(x[j]))
            if len(y)==0:
                exit
    return CLUSTER, y, len(CLUSTER)

def sort(array):
    """Sort the array by using quicksort"""

    less = []
    equal = []
    greater = []

    if len(array) > 1:
        pivot = array[0]
        for x in array:
            if x < pivot:
                less.append(x)
            elif x == pivot:
                equal.append(x)
            elif x > pivot:
                greater.append(x)
        # Don't forget to return something!
        return sort(less)+equal+sort(greater)  # Just use the + operator to join lists
    # Note that you want equal ^^^^^ not pivot
    else:  # You need to handle the part at the end of the recursion - when you only have one element in your array, just return the array.
        return array

def burbuja(A): #Bubble method sort
    for i in range(1,len(A)):
        for j in range(0,len(A)-i):
            if(A[j+1] < A[j]):
                aux=A[j]
                A[j]=A[j+1]
                A[j+1]=aux
    return A

def segundo_menor(numeros): # The second smallest element of an array
    if isinstance(numeros, list):
        if len(numeros) < 2:
            return None
        
        if len(numeros) == 2 and numeros[0] == numeros[1]:
            return numeros[0]

        duplicados = set()
        unicos = []

        for n in numeros:
            if n not in duplicados:
                duplicados.add(n)
                unicos.append(n)

        unicos.sort()
        return unicos[1]

    raise TypeError("El parámetro numeros debe ser una lista.")

def segundo_mayor(numeros): # The second biggest element of an array
    if isinstance(numeros, list):
        if len(numeros) < 2:
            return None
        
        if len(numeros) == 2 and numeros[0] == numeros[1]:
            return None

        duplicados = set()
        unicos = []

        for n in numeros:
            if n not in duplicados:
                duplicados.add(n)
                unicos.append(n)

        unicos.sort()
        return unicos[-2]

    raise TypeError("El parámetro numeros debe ser una lista.")

def burbuja_ARRAY(A,N): #Sort the array A[1] from smallest to largest and in the same way I sort the array A[0]
    for i in range(1,N):
        for j in range(0,N-i):
            if(A[1][j+1] < A[1][j]):
                aux=A[1][j+1]
                A[1][j+1]=A[1][j]
                A[1][j]=aux
                
                aux=A[0][j+1]
                A[0][j+1]=A[0][j]
                A[0][j]=aux

    return A

def f(x,a): # Logistic Map
    f = 1.0 - a*x*x
    return f

def vector(M0,X0,M,N,epsilon,a): # I define a function that allows me to change the Mi for each element of the array: array=M0@f(X0,a) 
    #Z=M0@f(X0,a) 
    Z=np.dot(M0,f(X0,a))
    for i in range (N):
        if M[i] == 0:
            Z[i]=0
        else:
            Z[i]=(epsilon/M[i])*Z[i]
    return Z

def Average(lst):
    s = sum(lst) / len(lst) 
    return s

def create_directory_if_not_exists(directory_name):
    # Check if the directory already exists
    if not os.path.exists(directory_name):
        # If it doesn't exist, create it
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created.")
    else:
        print(f"The directory '{directory_name}' already exists.")

## *Setting initial parameters*

In [3]:
N = 300 # Number of Nodes
T = 100 # Discrete Time Associated with Coupled Logistic Ma
a = 1.7 # Parameter defining the laminar and/or chaotic behavior of the system
delta = 10**-1 # Threshold of coherence function
cantidad_de_iteraciones = 10000 # Number of time iterations
epsilon = np.array([0.6]) # In this array, the different epsilon values to be calculated will be placed

In [4]:
for k in range(len(epsilon)):
    epsilon[k] = format(epsilon[k],".3f")
epsilon = list(epsilon)

NODOS=np.arange(0,N) #Array used to identify nodes one by one
MATRIX_ORDEN=np.empty((N,N),dtype=int) # Empty matrix that will orderly contain the states of the node

# *Creations of Folders*

In [5]:
for i in range(len(epsilon)):
    create_directory_if_not_exists('PLOTS/CLUSTER_SIZE/EPS_'+str(format(epsilon[i],".3f"))+'/')

    create_directory_if_not_exists('PLOTS/RED/EPS_' + str(format(epsilon[i],".3f")) +'/')

    create_directory_if_not_exists('PLOTS/STATES/EPS_' + str(format(epsilon[i],".3f")) +'/')

Directory 'PLOTS/CLUSTER_SIZE/EPS_0.600/' created.
Directory 'PLOTS/RED/EPS_0.600/' created.
Directory 'PLOTS/STATES/EPS_0.600/' created.


In [ ]:
for k in range(len(epsilon)):
   
    print("Epsilon:", epsilon[k])
    p = 0.06  # <k> = 18, p: probability of have a link between a node i and j 
    p_sort=random.random()  
    
    M0 = np.empty((N,N))  

    for i in range(N):
        p_sort=random.random()
        for j in range(N):
            p_sort=random.random()
            if p_sort < p: # With this, I ensure that the total number of connections is on the order of 5200
                M0[i][j] = 1
            else:
                M0[i][j] = 0

    for i in range(N):
        M0[i][i] = 0

    for i in range(N):
        for j in range(N):
            M0[i][j]= M0[j][i]

    M=[]

    for i in range(N):
        M.append(sum(M0[i])) # Number of neighbors in the i-th node
    print("<k> =", Average(M))

    outfile = open("PLOTS/CLUSTERING_EPS_" + str(format(epsilon[k],".3f")) + ".txt","w")
    outfile7 = open("PLOTS/CLUSTER_SIZE/EPS_"+ str(format(epsilon[k],".3f")) + "/CANTIDAD_CLUST_" + str(format(epsilon[k],".3f")) + ".txt","w")
    outfile8 = open("PLOTS/CLUSTER_SIZE/EPS_"+ str(format(epsilon[k],".3f")) + "/CANTIDAD_ELEM_CLUST_" + str(format(epsilon[k],".3f")) + ".txt","w")
    outfile9 = open("PLOTS/PATH_EPS_" + str(format(epsilon[k],".3f")) + ".txt","w")

    iteraciones=-1
    while iteraciones < cantidad_de_iteraciones:
        iteraciones = iteraciones + 1
        X0=np.random.uniform(low=-0.99999999999,high=1,size=N) # Initial Condition of the States: ARRAY OF ORDER N WITH N RANDOM ELEMENTS IN (-1,1)

        if iteraciones == 0:
            print("iteración: ", iteraciones)
            G = nx.from_numpy_array(M0)
            #--------------------------------States----------------------------------------------
            np.savetxt('PLOTS\STATES\EPS_' + str(format(epsilon[k],".3f")) +'\\X0_'+ str(format(epsilon[k],".3f")) + '_' + str(0) + '.txt', X0,  delimiter=',')
            #-------------------------------------------------------------------------------------
            #--------------------------------CLUSTERING-------------------------------------------
            c = nx.average_clustering(G)
            print("c:", c) 
            outfile.write(f"{c}\n")
            #-------------------------------------------------------------------------------------
            #-----------------------------------Path Length---------------------------------------
            length = nx.average_shortest_path_length(G)
            print("l:", length) 
            outfile9.write(f"{length}\n")
            #-------------------------------------------------------------------------------------
            #----------------------------------MATRIX------------------------------------------- 
            np.savetxt('PLOTS\RED\EPS_' + str(format(epsilon[k],".3f")) + '\\M0_'+ str(format(epsilon[k],".3f")) + '_' + str(iteraciones)+ '.txt', M0, fmt='%i',  delimiter=',')
            #-------------------------------------------------------------------------------------
            #----------------------------------CLUSTERS------------------------------------------- 
            lista=X0 # In order to calculate CLUSTER_SIZE
            ESTADOS_ORDENADOS = sort(list(lista))
            LONGITUD_ESTADOS=len(ESTADOS_ORDENADOS)
            s = LONGITUD_ESTADOS-1 #A PRIORI FIX: Start analyzing elements from the element LONGITUD_ESTADOS-1.
            #s The last element of the array
            #LONGITUD_ESTADOS: Variable size of the array

            Cantidad_clusters = 0

            while(s>0): # In order to calculate CLUSTER_SIZE

                for j in range(s-1,-1,-1):
                    if (abs(ESTADOS_ORDENADOS[j]-ESTADOS_ORDENADOS[s]) > delta):
                        Cantidad_elem_clusters=0
                        for q in range(s, j, -1):
                            ESTADOS_ORDENADOS.pop(q)
                            Cantidad_elem_clusters +=1
                        outfile8.write(f"{Cantidad_elem_clusters}\n")    
                        Cantidad_clusters += 1  
                        s=len(ESTADOS_ORDENADOS)-1

                    elif (j==0 and abs(ESTADOS_ORDENADOS[0]-ESTADOS_ORDENADOS[1]) > delta):
                        s=0
                        Cantidad_clusters += 1
                        Cantidad_elem_clusters=1 
                        outfile8.write(f"{Cantidad_elem_clusters}\n") 

                    elif (j==0 and abs(ESTADOS_ORDENADOS[s]-ESTADOS_ORDENADOS[j]) <= delta):
                        s=0
                        Cantidad_clusters += 1
                        Cantidad_elem_clusters=0
                        for q in range(s, j, -1):
                            ESTADOS_ORDENADOS.pop(q)
                        Cantidad_elem_clusters =len(ESTADOS_ORDENADOS)
                        outfile8.write(f"{Cantidad_elem_clusters}\n")  

            string=["--------------------"]
            outfile8.write(f"{string[0]}\n") # In order to calculate CLUSTER_SIZE
            outfile7.write(f"{Cantidad_clusters}\n")   # In order to calculate CLUSTER_SIZE

        # Now I start with the logistic map iteration!    

        for l in range(T):
            Y=(1.0-epsilon[k])*f(X0,a)+vector(M0,X0,M,N,epsilon[k],a) # Evaluation of the map_log_acop function in an array of order N
            X0=Y  # Array of N nodes in the j-th state
        STATES = Y

        i=random.randint(0,N-1) # I randomly choose a node after completing the T iterations 
        DIST=[] # I define an empty list where all distances will be stored
        DIST_MAX=[] # I define an empty list where the distances of the connected nodes will be stored

        for j in range(N):
            if i != j:  
                dij=abs(Y[i]-Y[j]) # I iterate over all nodes and measure the distance
                DIST.append(dij)
                if M0[i][j] == 1:
                    DIST_MAX.append(dij)
            elif i == j:
                flag=-1000
                DIST.append(flag) # If k == j, I don't measure the distance/add a flag

        dij1 = segundo_menor(DIST) # j1 ---> minimum, since the smallest is the flag == -1000
        index_list_min = DIST.index(dij1) # Among all nodes
        
        if len(DIST_MAX) != 0:
            dij2 = np.max(DIST_MAX) # j2 ---> maximum
            index_list_max = DIST.index(dij2) # AMONG NEIGHBOR NODES ONLY

        #-----------------------------CHANGE IN THE ADJACENCY MATRIX------------------------------------
        # dij1(T+1) ---> minimum
        # dij2(T+1) ---> maximum
        # If j1 is a neighbor of i ---> No changes
        # If j2 is a neighbor of i ---> The connection between i and j2 is replaced by the connection between i and j1
        # The node to which distances are calculated with the remaining nodes is the i-th node (i)
        # I check the position of the maximum and minimum indices and based on that determine how to change the 
        # elements of the system matrix.
        #--------------------------------------------------------------------------------------------------
        if len(DIST_MAX) != int(0):
            if M0[i][index_list_min] == 1:
                pass
            else: 
                M0[i][index_list_min] = 1
                M0[index_list_min][i] = 1
                M0[i][index_list_max] = 0
                M0[index_list_max][i] = 0

                M[i] = sum(M0[i])
                M[index_list_min] = sum(M0[index_list_min])
                M[index_list_max] = sum(M0[index_list_max])
        #---------------------------------------------------------------------------------------------------             
        else: # If the selected node it is disconnected I delete this node from the network, I will do this just two times in this program
            M0 = np.delete(M0,(i), axis = 0) # Delete of the row and column i of the adjacency matrix
            M0 = np.delete(M0,(i), axis = 1)
            N=N-1
            NODOS=np.arange(0,N) #Array used to identify nodes one by one
            MATRIX_ORDEN=np.empty((N,N),dtype=int) 
            X0=np.delete(X0,i)
            Y=np.delete(Y,i)
            M=np.delete(M,i)

            i=random.randint(0,N-1) # I randomly choose another node after completing the T iterations

            DIST=[] # I define an empty list where the distances will be stored
            DIST_MAX=[]
            for j in range(N):
                if i != j:  
                    dij=abs(Y[i]-Y[j]) # I iterate over all nodes and measure the distance
                    DIST.append(dij)
                    if M0[i][j] == 1:
                        DIST_MAX.append(dij)
                elif i == j:
                    flag=-1000
                    DIST.append(flag) # If k == j, I don't measure the distance/add a flag

            dij1 = segundo_menor(DIST) # j1 ---> minimum
            index_list_min = DIST.index(dij1) # AMONG ALL NODES
            
            if len(DIST_MAX) != 0:
                dij2 = np.max(DIST_MAX) # j2 ---> maximum
                index_list_max = DIST.index(dij2) # AMONG NEIGHBOR NODES ONLY

            if M0[i][index_list_min] == 1:
                pass
            else: 
                M0[i][index_list_min] = 1
                M0[index_list_min][i] = 1
                M0[i][index_list_max] = 0
                M0[index_list_max][i] = 0

                M[i] = sum(M0[i])
                M[index_list_min] = sum(M0[index_list_min])
                M[index_list_max] = sum(M0[index_list_max])    

        if iteraciones % 1000 == 0:
            print("iteración: ", iteraciones)
            G = nx.from_numpy_array(M0)
            #--------------------------------CLUSTERING-------------------------------------------
            c = nx.average_clustering(G)
            print("c:", c) 
            outfile.write(f"{c}\n")
            #-------------------------------------------------------------------------------------
            #-----------------------------------Path Length---------------------------------------
            length = nx.average_shortest_path_length(G)
            print("l:", length) 
            outfile9.write(f"{length}\n")
            #-------------------------------------------------------------------------------------
            #----------------------------------CLUSTERS------------------------------------------- 
            lista=STATES
            ESTADOS_ORDENADOS = sort(list(lista))
            LONGITUD_ESTADOS=len(ESTADOS_ORDENADOS)
            s = LONGITUD_ESTADOS-1 #FIJO A PRIORI Para empezar a analizar los elementos desde el elemento LONGITUD_ESTADOS-1
            #s Ultimo elemento del array
            #LONGITUD_ESTADOS tamaño variable del arrelgo
            
            Cantidad_clusters = 0

            while(s>0):
                for j in range(s-1,-1,-1):
                    if (abs(ESTADOS_ORDENADOS[j]-ESTADOS_ORDENADOS[s]) > delta):
                        Cantidad_elem_clusters = 0
                        for q in range(s, j, -1):
                            ESTADOS_ORDENADOS.pop(q)
                            Cantidad_elem_clusters +=1
                        outfile8.write(f"{Cantidad_elem_clusters}\n")    
                        Cantidad_clusters += 1  
                        s=len(ESTADOS_ORDENADOS)-1

                    elif (j==0 and abs(ESTADOS_ORDENADOS[0]-ESTADOS_ORDENADOS[1]) > delta):
                        s=0
                        Cantidad_clusters += 1
                        Cantidad_elem_clusters = 1 
                        outfile8.write(f"{Cantidad_elem_clusters}\n") 

                    elif (j==0 and abs(ESTADOS_ORDENADOS[s]-ESTADOS_ORDENADOS[j]) <= delta):
                        s=0
                        Cantidad_clusters += 1
                        Cantidad_elem_clusters = 0
                        for q in range(s, j, -1):
                            ESTADOS_ORDENADOS.pop(q)
                        Cantidad_elem_clusters = len(ESTADOS_ORDENADOS)
                        outfile8.write(f"{Cantidad_elem_clusters}\n")  

            string=["--------------------"]
            outfile8.write(f"{string[0]}\n")
            outfile7.write(f"{Cantidad_clusters}\n")   
            #---------------------------------MATRIX----------------------------------------------  
            np.savetxt('PLOTS\RED\EPS_' + str(format(epsilon[k],".3f")) + '\\M0_'+ str(format(epsilon[k],".3f")) + '_' + str(iteraciones)+ '.txt', M0, fmt='%i',  delimiter=',')
            #-----------------------------------STATES--------------------------------------------
            np.savetxt('PLOTS\STATES\EPS_' + str(format(epsilon[k],".3f")) + '\\X0_'+ str(format(epsilon[k],".3f")) + '_' + str(iteraciones)+ '.txt', STATES,  delimiter=',')
            #-------------------------------------------------------------------------------------